In [2]:
%%capture
!pip install wikidataintegrator

In [3]:
from wikidataintegrator import wdi_core

# Federated query

In [7]:
query = """
PREFIX wp:      <http://vocabularies.wikipathways.org/wp#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dc:      <http://purl.org/dc/elements/1.1/>
PREFIX wdt:     <http://www.wikidata.org/prop/direct/>
PREFIX wd:      <http://www.wikidata.org/entity/>
PREFIX ps:      <http://www.wikidata.org/prop/statement/>
PREFIX pq:      <http://www.wikidata.org/prop/qualifier/>
PREFIX p:       <http://www.wikidata.org/prop/>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd:       <http://www.bigdata.com/rdf#>

SELECT DISTINCT ?gene ?geneLabel ?wd_pathwayLabel ?tneighbour  ?interactionType where {
  ?gene wdt:P2293 ?diseaseGA .        # gene has genetic association
  ?diseaseGA wdt:P279* wd:Q3286546 .  # to a type of respiratory system disease
  #?gene wdt:P31 wd:Q7187 .     # gene is subclass of "gene"

  ?gene wdt:P688 ?protein ; # gene encodes a protein
        wdt:P2888 ?gene_uri ;
        wdt:P594 ?ensemblID .
  ?protein p:P681 ?s .                             # protein's cell component statement
  ?s ps:P681 ?cp .                               # get statement value
  FILTER NOT EXISTS {?s pq:P459 wd:Q23190881 .}  # determination method is not IEA
  # ?cp wdt:P279*|wdt:P361* wd:Q14349455 .         # statement value is 'part of' or 'subclass of' membrane

  ?wd_pathway wdt:P31 wd:Q4915012 ;               # instance of a biological pathway
             wdt:P2888 ?pwIri ;                     # Mapping to Wikipathways IRI
             wdt:P527 ?gene .
  SERVICE <http://sparql.wikipathways.org/sparql> {

      ?wp_pathway dc:identifier ?pwIri ;
                wp:ontologyTag ?pathwayTags .
    ?pathwayTags rdfs:label ?tags .
    ?interaction dcterms:isPartOf ?wp_pathway ;
                 a wp:Interaction, ?interactionType ;
                 wp:participants ?gene_uri .
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 10
"""
wdi_core.WDFunctionsEngine.execute_sparql_query(query, as_dataframe=True)

,gene,geneLabel,wd_pathwayLabel,interactionType
0,http://www.wikidata.org/entity/Q14912501,FTO,FTO Obesity Variant Mechanism,http://vocabularies.wikipathways.org/wp#Binding
1,http://www.wikidata.org/entity/Q14912501,FTO,FTO Obesity Variant Mechanism,http://vocabularies.wikipathways.org/wp#Direct...
2,http://www.wikidata.org/entity/Q14912501,FTO,FTO Obesity Variant Mechanism,http://vocabularies.wikipathways.org/wp#Intera...
3,http://www.wikidata.org/entity/Q14914344,TNXB,VEGFA-VEGFR2 Signaling Pathway,http://vocabularies.wikipathways.org/wp#Direct...
4,http://www.wikidata.org/entity/Q14914344,TNXB,VEGFA-VEGFR2 Signaling Pathway,http://vocabularies.wikipathways.org/wp#Intera...
5,http://www.wikidata.org/entity/Q14907362,SCGB1A1,Prostaglandin Synthesis and Regulation,http://vocabularies.wikipathways.org/wp#Direct...
6,http://www.wikidata.org/entity/Q14907362,SCGB1A1,Prostaglandin Synthesis and Regulation,http://vocabularies.wikipathways.org/wp#Inhibi...
7,http://www.wikidata.org/entity/Q14907362,SCGB1A1,Prostaglandin Synthesis and Regulation,http://vocabularies.wikipathways.org/wp#Intera...
8,http://www.wikidata.org/entity/Q14886904,GLI3,Endochondral Ossification,http://vocabularies.wikipathways.org/wp#Direct...
9,http://www.wikidata.org/entity/Q14886904,GLI3,Endochondral Ossification,http://vocabularies.wikipathways.org/wp#Inhibi...


# Queries in series

In [52]:
wikidata_query = """
PREFIX wp:      <http://vocabularies.wikipathways.org/wp#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dc:      <http://purl.org/dc/elements/1.1/>
PREFIX wdt:     <http://www.wikidata.org/prop/direct/>
PREFIX wd:      <http://www.wikidata.org/entity/>
PREFIX ps:      <http://www.wikidata.org/prop/statement/>
PREFIX pq:      <http://www.wikidata.org/prop/qualifier/>
PREFIX p:       <http://www.wikidata.org/prop/>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd:       <http://www.bigdata.com/rdf#>

SELECT DISTINCT ?gene ?geneLabel ?gene_uri ?wd_pathwayLabel ?pwIri where {
  ?gene wdt:P2293 ?diseaseGA .        # gene has genetic association
  ?diseaseGA wdt:P279* wd:Q3286546 .  # to a type of respiratory system disease
  #?gene wdt:P31 wd:Q7187 .     # gene is subclass of "gene"

  ?gene wdt:P688 ?protein ; # gene encodes a protein
        wdt:P2888 ?gene_uri ;
        wdt:P594 ?ensemblID .
  ?protein p:P681 ?s .                             # protein's cell component statement
  ?s ps:P681 ?cp .                               # get statement value
  FILTER NOT EXISTS {?s pq:P459 wd:Q23190881 .}  # determination method is not IEA
  ?cp wdt:P279*|wdt:P361* wd:Q14349455 .         # statement value is 'part of' or 'subclass of' membrane

  ?wd_pathway wdt:P31 wd:Q4915012 ;               # instance of a biological pathway
             wdt:P2888 ?pwIri ;                     # Mapping to Wikipathways IRI
             wdt:P527 ?gene .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}         
"""
df = wdi_core.WDFunctionsEngine.execute_sparql_query(query=wikidata_query, as_dataframe=True)
df["gene_uri"] = "<"+df['gene_uri'].astype(str)+">"
df

,gene,gene_uri,pwIri,geneLabel,wd_pathwayLabel
0,http://www.wikidata.org/entity/Q14865565,<http://identifiers.org/ncbigene/1956>,http://identifiers.org/wikipathways/WP382,EGFR,MAPK Signaling Pathway
1,http://www.wikidata.org/entity/Q14865565,<http://identifiers.org/ncbigene/1956>,http://identifiers.org/wikipathways/WP673,EGFR,ErbB Signaling Pathway
2,http://www.wikidata.org/entity/Q14865565,<http://identifiers.org/ncbigene/1956>,http://identifiers.org/wikipathways/WP3931,EGFR,ESC Pluripotency Pathways
3,http://www.wikidata.org/entity/Q14865565,<http://identifiers.org/ncbigene/1956>,http://identifiers.org/wikipathways/WP306,EGFR,Focal Adhesion
4,http://www.wikidata.org/entity/Q14865565,<http://identifiers.org/ncbigene/1956>,http://identifiers.org/wikipathways/WP3611,EGFR,Photodynamic therapy-induced AP-1 survival sig...
...,...,...,...,...,...
245,http://www.wikidata.org/entity/Q18033785,<http://identifiers.org/ncbigene/9049>,http://identifiers.org/wikipathways/WP2586,AIP,Aryl Hydrocarbon Receptor
246,http://www.wikidata.org/entity/Q18028659,<http://identifiers.org/ncbigene/4036>,http://identifiers.org/wikipathways/WP1533,LRP2,Vitamin B12 Metabolism
247,http://www.wikidata.org/entity/Q18028659,<http://identifiers.org/ncbigene/4036>,http://identifiers.org/wikipathways/WP2806,LRP2,Human Complement System
248,http://www.wikidata.org/entity/Q17910399,<http://identifiers.org/ncbigene/1439>,http://identifiers.org/wikipathways/WP4538,CSF2RB,Regulatory circuits of the STAT3 signaling pat...


In [53]:
second_query = """
PREFIX wp:      <http://vocabularies.wikipathways.org/wp#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dc:      <http://purl.org/dc/elements/1.1/>
PREFIX wdt:     <http://www.wikidata.org/prop/direct/>
PREFIX wd:      <http://www.wikidata.org/entity/>
PREFIX ps:      <http://www.wikidata.org/prop/statement/>
PREFIX pq:      <http://www.wikidata.org/prop/qualifier/>
PREFIX p:       <http://www.wikidata.org/prop/>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd:       <http://www.bigdata.com/rdf#>

SELECT DISTINCT ?interactionType ?interactionTypeLabel where {{
  VALUES ?gene_uri {{ {values} }}
  SERVICE <http://sparql.wikipathways.org/sparql> {{

    ?wp_pathway dc:identifier ?pwIri ;
                wp:ontologyTag ?pathwayTags .
    ?pathwayTags rdfs:label ?tags .
    ?interaction dcterms:isPartOf ?wp_pathway ;
                 a wp:Interaction, ?interactionType ;
                 wp:participants ?gene_uri .
    ?interactionType rdfs:label ?interactionTypeLabel .
  }}
}}
""".format(values=" ".join(set(df["gene_uri"].tolist())))

wdi_core.WDFunctionsEngine.execute_sparql_query(query = second_query, as_dataframe=True)

,interactionTypeLabel,interactionType
0,DirectedInteraction,http://vocabularies.wikipathways.org/wp#Direct...
1,Transcription Translation,http://vocabularies.wikipathways.org/wp#Transc...
2,ComplexBinding,http://vocabularies.wikipathways.org/wp#Comple...
3,Interaction,http://vocabularies.wikipathways.org/wp#Intera...
4,Binding,http://vocabularies.wikipathways.org/wp#Binding
5,Inhibition,http://vocabularies.wikipathways.org/wp#Inhibi...


In [54]:
third_query = """
PREFIX wp:      <http://vocabularies.wikipathways.org/wp#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dc:      <http://purl.org/dc/elements/1.1/>
PREFIX wdt:     <http://www.wikidata.org/prop/direct/>
PREFIX wd:      <http://www.wikidata.org/entity/>
PREFIX ps:      <http://www.wikidata.org/prop/statement/>
PREFIX pq:      <http://www.wikidata.org/prop/qualifier/>
PREFIX p:       <http://www.wikidata.org/prop/>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd:       <http://www.bigdata.com/rdf#>

SELECT DISTINCT ?gene_uri ?neighbour ?interactionType ?interactionTypeLabel where {{
  VALUES ?gene_uri {{ {values} }}
  SERVICE <http://sparql.wikipathways.org/sparql> {{
    ?wp_pathway dc:identifier ?pwIri ;
                wp:ontologyTag ?pathwayTags .
    ?pathwayTags rdfs:label ?tags .
    ?interaction dcterms:isPartOf ?wp_pathway ;
                 a wp:Interaction, ?interactionType ;
                 wp:participants ?gene_uri, ?neighbour .
    ?interactionType rdfs:label ?interactionTypeLabel .
  }}
}}
""".format(values=" ".join(set(df["gene_uri"].tolist())))

wdi_core.WDFunctionsEngine.execute_sparql_query(query = third_query, as_dataframe=True)

,gene_uri,neighbour,interactionTypeLabel,interactionType
0,http://identifiers.org/ncbigene/4855,http://identifiers.org/ncbigene/2078,DirectedInteraction,http://vocabularies.wikipathways.org/wp#Direct...
1,http://identifiers.org/ncbigene/4855,http://identifiers.org/ncbigene/2078,Transcription Translation,http://vocabularies.wikipathways.org/wp#Transc...
2,http://identifiers.org/ncbigene/4855,http://identifiers.org/ncbigene/2078,Interaction,http://vocabularies.wikipathways.org/wp#Intera...
3,http://identifiers.org/ncbigene/919,http://identifiers.org/ncbigene/3932,DirectedInteraction,http://vocabularies.wikipathways.org/wp#Direct...
4,http://identifiers.org/ncbigene/919,http://identifiers.org/ncbigene/3932,Interaction,http://vocabularies.wikipathways.org/wp#Intera...
...,...,...,...,...
370,http://identifiers.org/ncbigene/8795,http://identifiers.org/ncbigene/7133,Binding,http://vocabularies.wikipathways.org/wp#Binding
371,http://identifiers.org/ncbigene/2066,http://identifiers.org/ncbigene/2066,DirectedInteraction,http://vocabularies.wikipathways.org/wp#Direct...
372,http://identifiers.org/ncbigene/8795,http://identifiers.org/ncbigene/8795,DirectedInteraction,http://vocabularies.wikipathways.org/wp#Direct...
373,http://identifiers.org/ncbigene/8795,http://identifiers.org/ncbigene/4790,DirectedInteraction,http://vocabularies.wikipathways.org/wp#Direct...
